In [1]:
using Solve
using BenchmarkTools
using DifferentialEquations

In [10]:
# These were all run on my office desktop machine:
# 8GB of RAM
# 4 cores, 1 thread/core, Intel i5-4590
# GPU is not used for these simulations

In [3]:
# parameters
duration = 1 # seconds
nruns = 20 # number of runs
nsave = 100 # number of save points during the run
B0 = crit_params["B0"]
B1 = crit_params["B1"]

0.40289106454828055

In [4]:
# Baseline, spin dressing simulation. Using my defaults:
# reltol=abstol=1e-10
# The integration algorithm I use is Vern9()
@benchmark run_simulations(duration, nruns; B0=B0, B1=B1, nsave=nsave,
            )

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  460.308 ms … 476.677 ms  ┊ GC (min … max): 0.00% … 1.00%
 Time  (median):     469.901 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   469.307 ms ±   5.754 ms  ┊ GC (mean ± σ):  0.09% ± 0.30%

  █    █     █         █         █   █        █ █      █   █  █  
  █▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁█▁▁▁█▁▁█ ▁
  460 ms           Histogram: frequency by time          477 ms <

 Memory estimate: 10.80 MiB, allocs estimate: 38569.

In [5]:
#Same as baseline, but use EnsembleSerial instead of EnsembleThreads
@benchmark run_simulations(duration, nruns; B0=B0, B1=B1, nsave=nsave,
            ensemblehandling=EnsembleSerial())

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  453.841 ms … 470.777 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     459.942 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   461.793 ms ±   5.562 ms  ┊ GC (mean ± σ):  0.10% ± 0.32%

  █     █     █   █  █ █              █  █     █        █     █  
  █▁▁▁▁▁█▁▁▁▁▁█▁▁▁█▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁█ ▁
  454 ms           Histogram: frequency by time          471 ms <

 Memory estimate: 10.80 MiB, allocs estimate: 38569.

In [6]:
#Same as baseline, but increase tolerances 1e-10->1e-11
@benchmark run_simulations(duration, nruns; B0=B0, B1=B1, nsave=nsave,
            abstol=1e-11,
            reltol=1e-11
            )

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  565.638 ms … 598.398 ms  ┊ GC (min … max): 0.00% … 1.04%
 Time  (median):     579.812 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   578.072 ms ±  11.361 ms  ┊ GC (mean ± σ):  0.12% ± 0.35%

  ██  █ █                   █     ██       █                  █  
  ██▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁██▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  566 ms           Histogram: frequency by time          598 ms <

 Memory estimate: 10.80 MiB, allocs estimate: 38565.

In [7]:
# Same as baseline, but add some white noise along B1 direction
noiserms = crit_params["B1"] * 1e-3
noiserate = 5e3 #Hz
noise_iterator = filterednoise(noiserms, duration, noiserate)
@benchmark run_simulations(duration, nruns; B0=B0, B1=B1, nsave=nsave,
            Bxfuncs=noise_iterator
            )

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  837.783 ms … 853.838 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     844.637 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   845.283 ms ±   5.939 ms  ┊ GC (mean ± σ):  0.07% ± 0.17%

  █          █        █           █             █             █  
  █▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  838 ms           Histogram: frequency by time          854 ms <

 Memory estimate: 17.79 MiB, allocs estimate: 35668.

In [8]:
# Same as baseline, but add some linear gradients
grad_matrix = [0 -1 0;
                1 0 0;
                0 0 1]
gradients = grad_matrix * B1 * 1e-3 # 1% gradients per meter
Efield = [0 0 0]
noiserate = 5e3
noise_iterator = spatialnoise(gradients, Efield, duration, noiserate)
@benchmark run_simulations(duration, nruns; B0=B0, B1=B1, nsave=nsave,
            Bfuncs=noise_iterator
            )

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  878.568 ms … 896.905 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     886.708 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   887.744 ms ±   6.671 ms  ┊ GC (mean ± σ):  0.06% ± 0.14%

  █                 █ █            █               █          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█ ▁
  879 ms           Histogram: frequency by time          897 ms <

 Memory estimate: 25.98 MiB, allocs estimate: 32981.

In [9]:
#Same as baseline, but do free precession
@benchmark run_simulations(duration, nruns; B0=B0, B1=0, nsave=nsave)

BenchmarkTools.Trial: 81 samples with 1 evaluation.
 Range (min … max):  60.844 ms … 71.904 ms  ┊ GC (min … max): 0.00% … 10.41%
 Time  (median):     62.034 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   62.452 ms ±  1.879 ms  ┊ GC (mean ± σ):  0.43% ±  1.97%

   ▄█    ▃  ▄                                                  
  ▄██▅▄▅▆█▃▄██▄▁▁▆▇▁▄▁▁▃▁▁▃▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  60.8 ms         Histogram: frequency by time          70 ms <

 Memory estimate: 4.28 MiB, allocs estimate: 32666.